In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Params soft-tissue
tissue_layers = [
    {"name": "Dermis", "thickness": 0.065, "mu_a": 227, "mu_s": 0.02, "g": 0.9},
    {"name": "Fat", "thickness": 0.3, "mu_a": 185.7, "mu_s": 1.07, "g": 0.89},
    {"name": "Muscle", "thickness": 0.3, "mu_a": 63.3, "mu_s": 0.7, "g": 0.89}
]

# Radiation parameters
wavelength = 637e-9# m (637 nm)
source_power = 0.8 # W
source_diameter = 1e-3 # m
distance_to_skin = 1e-3 # m
n_photons = 5000000
z_nerve = 0.85e-3 # m

# Semiconductor parameters
Eg1, Eg2 = 2.16, 2.4 # eV, band gap of H2Pc and PTCDI
chi1, chi2 = 3.23, 3.09 # eV, electron affinity
mu_n1, mu_n2 = 0.1e-4, 0.1e-4 # m²/(V s), electron mobility
mu_p1, mu_p2 = 0.1e-4, 0.12e-4 # m²/(V s), hole mobility
epsilon1, epsilon2 = 2, 2 # Dielectric constants
L1, L2 = 30e-9, 30e-9 # m, layer thickness
phi1, phi2 = 5.3852, 3.09 # eV, electrode work function
GX = 1e19 # m⁻²s⁻¹, pair generation rate
k_diss = 1e3 # s⁻¹, dissociation rate
k_rec = 1e3 # s⁻¹, recombination rate
q = 1.602e-19 # C, electron charge
k_B = 1.381e-23 # J/K, Boltzmann constant
T = 293.15 # K, temperature
Ne_v = 1e26 # m⁻³, density of states

In [ ]:
def monte_carlo_light_propagation():
    photons = np.zeros((n_photons, 3))
    weights = np.ones(n_photons)
    direction = np.array([0, 0, 1], dtype=float)
    mu_t = np.array([layer["mu_a"] + layer["mu_s"] for layer in tissue_layers]) / 100
    g = np.array([layer["g"] for layer in tissue_layers])
    z_boundaries = np.cumsum([0] + [layer["thickness"] for layer in tissue_layers]) / 100
    x_grid = np.linspace(-0.3, 0.3, 100)
    z_grid = np.linspace(0, z_boundaries[-1], 100)
    intensity = np.zeros((len(z_grid), len(x_grid)))

    for i in range(n_photons):
        r = np.sqrt(np.random.random()) * (source_diameter / 2)
        theta = 2 * np.pi * np.random.random()
        pos = np.array([r * np.cos(theta), r * np.sin(theta), distance_to_skin], dtype=float)
        w = weights[i]
        alive = True
        while alive:
            layer_idx = np.searchsorted(z_boundaries, pos[2], side="right") - 1
            if layer_idx >= len(tissue_layers) or pos[2] < 0:
                alive = False
                continue
            mu_a = tissue_layers[layer_idx]["mu_a"] / 100  # cm⁻¹ to m⁻¹
            mu_s = tissue_layers[layer_idx]["mu_s"] / 100
            g_layer = tissue_layers[layer_idx]["g"]
            s = -np.log(np.random.random()) / (mu_a + mu_s)
            pos += s * direction
            delta_w = w * mu_a / (mu_a + mu_s)
            x_idx = np.argmin(np.abs(x_grid - pos[0]))
            z_idx = np.argmin(np.abs(z_grid - pos[2]))
            if 0 <= x_idx < len(x_grid) and 0 <= z_idx < len(z_grid):
                intensity[z_idx, x_idx] += delta_w
            w *= mu_s / (mu_a + mu_s)

            if g_layer == 0:
                cos_theta = 2 * np.random.random() - 1
            else:
                cos_theta = (1 + g_layer**2 - ((1 - g_layer**2) / (1 - g_layer + 2 * g_layer * np.random.random()))**2) / (2 * g_layer)
            sin_theta = np.sqrt(1 - cos_theta**2)
            phi = 2 * np.pi * np.random.random()
            direction_new = np.array([sin_theta * np.cos(phi),
                sin_theta * np.sin(phi),
                cos_theta
            ], dtype=float)

            if abs(direction[2]) < 0.99999:
                sin_psi = np.sqrt(1 - direction[2]**2)
                cos_psi = direction[2]
                cos_phi = direction[0] / sin_psi
                sin_phi = direction[1] / sin_psi
                direction = np.array([
                    cos_theta * direction[0] + sin_theta * (cos_phi * cos_psi * cos_phi - sin_phi * sin_psi),
                    cos_theta * direction[1] + sin_theta * (sin_phi * cos_psi * cos_phi + cos_phi * sin_psi),
                    -sin_theta * sin_psi * cos_phi + cos_theta * cos_psi
                ], dtype=float)
            else:
                direction = direction_new
            if pos[2] > z_boundaries[-1] or w < 1e-4:
                alive = False
    intensity /= (n_photons * (source_diameter / 2) ** 2 * np.pi)
    intensity *= source_power
    return x_grid, z_grid, intensity

def calculate_photo_emf(intensity, z_nerve, z_grid):
    z_idx = np.argmin(np.abs(z_grid - z_nerve))
    I_nerve = np.mean(intensity[z_idx, :])
    print(f"Intensity at nerve depth: {I_nerve:.2e} W/m²")
    h = 6.626e-34
    c = 3e8
    photon_energy = h * c / wavelength
    photon_flux = I_nerve / photon_energy
    G = (photon_flux * (1 - np.exp(-k_diss / k_rec)))
    J_photo = q * G * (L1 + L2)
    print(f"Photocurrent density: {J_photo:.2e} A/m²")
    dark_current = q * k_rec * Ne_v * (L1 + L2)
    V_oc = (k_B * T / q) * np.log(1 + J_photo / dark_current)
    return V_oc

def semiconductor_model():
    x = np.linspace(0, (L1 + L2) * 1e9, 100)
    x_m = x * 1e-9
    dx = x_m[1] - x_m[0]
    Ec = np.zeros_like(x_m)
    Ev = np.zeros_like(x_m)
    Ec[x_m <= L1] = -chi1
    Ec[x_m > L1] = -chi2
    Ev[x_m <= L1] = -chi1 - Eg1
    Ev[x_m > L1] = -chi2 - Eg2
    n0 = Ne_v * np.exp(-(Ec + phi1) / (k_B * T / q))
    p0 = Ne_v * np.exp(-(-Ev - phi1) / (k_B * T / q))
    n = np.zeros_like(x_m)
    p = np.zeros_like(x_m)
    Gx = GX * np.ones_like(x_m)
    for i in range(1, len(x_m) - 1):
        dn_dx = (n[i+1] - n[i-1]) / (2 * dx)
        dp_dx = (p[i+1] - p[i-1]) / (2 * dx)
        n[i] = n0[i] + dx * (mu_n1 * dn_dx - k_rec * n[i] * p[i] + Gx[i]) if x_m[i] <= L1 else \
               n0[i] + dx * (mu_n2 * dn_dx - k_rec * n[i] * p[i] + Gx[i])
        p[i] = p0[i] + dx * (mu_p1 * dp_dx - k_rec * n[i] * p[i] + Gx[i]) if x_m[i] <= L1 else \
               p0[i] + dx * (mu_p2 * dp_dx - k_rec * n[i] * p[i] + Gx[i])
    n[0] = n0[0]
    n[-1] = n0[-1]
    p[0] = p0[0]
    p[-1] = p0[-1]

    doping = p - n
    epsilon_0 = 8.854e-12
    epsilon = np.ones_like(x_m) * epsilon1
    epsilon[x_m > L1] = epsilon2
    rho = q * (p - n)
    V = np.zeros_like(x_m)
    for i in range(1, len(x_m) - 1):
        V[i] = (V[i+1] + V[i-1] - dx**2 * rho[i] / (epsilon[i] * epsilon_0)) / 2
    V[0] = 0
    V[-1] = 0

    return x, doping, Ec, Ev, n, p, V

x_grid, z_grid, intensity = monte_carlo_light_propagation()
V_oc = calculate_photo_emf(intensity, z_nerve, z_grid)

In [ ]:
plt.figure(figsize=(8, 6))
log_intensity = np.log10(intensity + 1e-10)
plt.imshow(log_intensity, extent=[x_grid[0] * 100, x_grid[-1] * 100, z_grid[-1] * 100, z_grid[0] * 100],
cmap='hot', aspect='auto')
plt.colorbar(label='log10(Φ)')
plt.xlabel('X [cm]')
plt.ylabel('Z [cm]')
plt.title('Optical intensity distribution in rat three-layer tissue model')
plt.axhline(y=z_nerve * 100, color='white', linestyle='--', label='Nerve depth (0.85 mm)')
plt.legend()
#plt.savefig('light_distribution.png')

x, doping, Ec, Ev, n, p, V = semiconductor_model()

plt.figure(figsize=(8, 6))
plt.plot(x, doping, label='Pure doping')
plt.xlabel('Position [nm]')
plt.ylabel('Concentration [m⁻³]')
plt.title('Total pure doping')
plt.grid(True)
plt.legend()
#plt.savefig('doping.png')

plt.figure(figsize=(8, 6))
plt.plot(x, Ec, label='Ec (conduction band)')
plt.plot(x, Ev, label='Ev (valence band)')
plt.xlabel('Position [nm]')
plt.ylabel('Energy [eV]')
plt.title('Energy band diagram')
plt.grid(True)
plt.legend()
#plt.savefig('band_diagram.png')

plt.figure(figsize=(8, 6))
plt.plot(x, n, label='Electrons')
plt.xlabel('Position [nm]')
plt.ylabel('Concentration [m⁻³]')
plt.title('Electron Concentration')
plt.grid(True)
plt.legend()
#plt.savefig('electron_concentration.png')

plt.figure(figsize=(8, 6))
plt.plot(x, p, label='Holes')
plt.xlabel('Position [nm]')
plt.ylabel('Concentration [m⁻³]')
plt.title('Hole concentration')
plt.grid(True)
plt.legend()
#plt.savefig('pit_concentration.png')

plt.figure(figsize=(8, 6))
plt.plot(x, V, label='Voltage')
plt.xlabel('Position [nm]')
plt.ylabel('Voltage [V]')
plt.title('Voltage distribution for red light emission 625 nm')
plt.grid(True)
plt.legend()
#plt.savefig('stress_distribution.png')